In [1]:
# Ignore SQLITE warnings related to Decimal numbers in the housing database.
import warnings
warnings.filterwarnings('ignore')

In [2]:
# Import Dependencies.
import pandas as pd
import numpy as np
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, inspect, func

In [3]:
# Create an engine for the housing.sqlite database.
engine = create_engine("sqlite:///../Resources/housingUpdated.sqlite")

In [4]:
# Reflect Database into ORM classes.
Base = automap_base()
Base.prepare(engine, reflect=True)

In [5]:
# Create a database session object.
session = Session(engine)

In [6]:
# Get the table names of the database.
inspector = inspect(engine)
inspector.get_table_names()

['listings']

In [7]:
# Get the names and types of the columns for the table listings.
columns = inspector.get_columns("listings")
for c in columns:
    print(c["name"], c["type"])


id INTEGER
address VARCHAR(255)
price INTEGER
home_type VARCHAR(255)
bedrooms INTEGER
bathrooms FLOAT
square_feet INTEGER
built INTEGER
lot_size FLOAT
neighborhood VARCHAR(255)
county VARCHAR(255)
city VARCHAR(255)
zipcode INTEGER
high_school VARCHAR(255)
middle_school VARCHAR(255)
elementary_school VARCHAR(255)


In [8]:
# Save a reference to the listings table as "Listings".
Listings = Base.classes.listings

In [9]:
# Count the number of entries in the listings table.
listings_count = session.query(Listings).count()
listings_count

2064

In [10]:
# Count the number of distinct home types found in the listings table.
home_types_count = session.query(Listings.home_type).distinct().count()
home_types_count
# Given the sheer number of categories, this seems a good variable to drop...

73

In [11]:
# City, county, and neighborhood would seem to be specified by zip code. Count the number of distinct zip codes.
zipcode_count = session.query(Listings.zipcode).distinct().count()
zipcode_count

33

In [12]:
# Count the number of schools for each level.
hs_count = session.query(Listings.high_school).distinct().count()
ms_count = session.query(Listings.middle_school).distinct().count()
es_count = session.query(Listings.elementary_school).distinct().count()
hs_count, ms_count, es_count
# There are an awful lot of these, these categories may need to be removed when training models...

(25, 62, 117)

In [13]:
# Examine the values in high_school for potentially erroneous data.
high_school_values = session.query(Listings.high_school).distinct().all()
high_school_values

[('Current Price:'),
 ('Reynolds'),
 ('Centennial'),
 ('David Douglas'),
 ('Other'),
 ('Lincoln'),
 ('Jefferson'),
 ('Sunset'),
 ('Beaverton'),
 ('Roosevelt'),
 ('Wilson'),
 ('Madison'),
 ('Westview'),
 ('Franklin'),
 ('Cleveland'),
 ('Tualatin'),
 ('Grant'),
 ('Parkrose'),
 ('Tigard'),
 ('Milwaukie'),
 ('Southridge'),
 ('Liberty'),
 ('Riverdale'),
 ('Scappoose'),
 ('Lake Oswego')]

In [14]:
# Remove rows where high school is listed as "Current Price:".
deletion = Listings.__table__.delete().where(Listings.high_school=="Current Price:")
engine.execute(deletion)

In [15]:
# Check the high school values again to confirm deletion.
high_school_values = session.query(Listings.high_school).distinct().all()
high_school_values

[('Reynolds'),
 ('Centennial'),
 ('David Douglas'),
 ('Other'),
 ('Lincoln'),
 ('Jefferson'),
 ('Sunset'),
 ('Beaverton'),
 ('Roosevelt'),
 ('Wilson'),
 ('Madison'),
 ('Westview'),
 ('Franklin'),
 ('Cleveland'),
 ('Tualatin'),
 ('Grant'),
 ('Parkrose'),
 ('Tigard'),
 ('Milwaukie'),
 ('Southridge'),
 ('Liberty'),
 ('Riverdale'),
 ('Scappoose'),
 ('Lake Oswego')]

In [16]:
# Examine the values for lot_size.
lot_size_values = session.query(Listings.lot_size).distinct().all()
lot_size_values

[(None),
 (0.07),
 (0.1),
 (0.16),
 (0.06),
 (0.15),
 (0.22),
 (0.17),
 (0.11),
 (0.04),
 (0.03),
 (0.02),
 (0.32),
 (0.21),
 (0.37),
 (0.53),
 (0.18),
 (0.13),
 (0.19),
 (0.39),
 (0.23),
 (0.3),
 (0.25),
 (0.35),
 (0.14),
 (0.12),
 (0.33),
 (0.09),
 (0.05),
 (0.2),
 (0.28),
 (0.5),
 (0.29),
 (0.24),
 (0.27),
 (1.09),
 (0.31),
 (0.34),
 (0.47),
 (0.26),
 (0.52),
 (0.7),
 (0.08),
 (0.57),
 (0.36),
 (0.63),
 (0.95),
 (2.1),
 (0.49),
 (0.48),
 (2.07),
 (0.45),
 (1.3),
 (0.01),
 (0.38),
 (0.43),
 (0.59),
 (0.85),
 (0.42),
 (0.56),
 (0.66),
 (1.78),
 (0.71),
 (0.77),
 (1.29),
 (5.04),
 (0.46),
 (0.55),
 (0.4),
 (15.23),
 (0.69),
 (0.76),
 (0.88),
 (3.12),
 (0.44),
 (0.97),
 (3.9),
 (0.83),
 (1.0),
 (0.58),
 (0.51),
 (0.8),
 (0.61),
 (0.75),
 (3.55),
 (0.64),
 (5.0),
 (2.08),
 (2.01),
 (0.41),
 (0.89),
 (1.62),
 (0.94),
 (1.37),
 (0.54),
 (1.12),
 (0.78),
 (2.86),
 (1.68),
 (4.85),
 (1.06),
 (0.73),
 (0.9),
 (7.44),
 (0.79),
 (1.22),
 (2.0),
 (25.25),
 (1.03),
 (72.71),
 (3.06),
 (1.31),
 (0

In [17]:
# Examine the values for home_type.
home_type_values = session.query(Listings.home_type).distinct().all()
home_type_values

[('Manufactured - Double Wide Manufact'),
 ('Floating Home - Contemporary'),
 ('Floating Home - Cabin'),
 ('Condo - Traditional'),
 ('Condo - Common Wall'),
 ('Condo - Other'),
 ('Single Family - Bungalow'),
 ('Condo - 1 Story'),
 ('Condo - Contemporary'),
 ('Condo - FourSquare'),
 ('Condo - Studio'),
 ('Condo - Tri Level'),
 ('Floating Home - Bungalow'),
 ('Floating Home - Manufactured Home'),
 ('Condo - 2 Story'),
 ('Floating Home - 2 Story'),
 ('Floating Home - 1 Story'),
 ('Manufactured - Manufactured Home'),
 ('Single Family - Ranch'),
 ('Condo - Mediterranean/Missio'),
 ('Condo - Detached Condo'),
 ('Condo - Craftsman'),
 ('Single Family - Townhouse'),
 ('Single Family - Traditional'),
 ('Condo - Townhouse'),
 ('Floating Home - Custom Style'),
 ('Condo - Row House'),
 ('Floating Home - Cottage'),
 ('Single Family - 1 Story'),
 ('Floating Home - Traditional'),
 ('Condo - Ranch'),
 ('Single Family - Craftsman'),
 ('Condo - Mid-Century Modern'),
 ('Condo - Tudor'),
 ('Single Family 

In [18]:
# Close the session.
session.close()